In [1]:
import re
import pandas as pd
import os
import convertTxtToDFFns

In [2]:
with open('./../charts/LA-1.4.19.txt') as file:
    ex1 = file.readlines() #import a txt file
with open('./../charts/LP-1.13.19.txt') as file:
    ex2 = file.readlines() #import another
with open('./../charts/MVR-1.2.19.txt') as file:
    ex3 = file.readlines() #import another
with open('./../charts/SUN-1.12.19.txt') as file:
    ex4 = file.readlines() #import another
with open('./../charts/belmont.txt') as file:
    belmont = file.readlines() #import another

In [3]:
test = convertTxtToDFFns.parseFullDay(ex4)

In [4]:
test    

,trackName,month,day,year,raceNum,breed,distance,surface,segment1,segment2,segment3,segment4,segment5,segments
0,SUN,1,12,2019,1,Thoroughbred,OneMileAndSeventyYards,Dirt,Start,1/4,1/2,3/4,Str,5
0,SUN,1,12,2019,2,Thoroughbred,SixAndOneHalfFurlongs,Dirt,Start,1/4,1/2,Str,,4
0,SUN,1,12,2019,3,Thoroughbred,SixFurlongs,Dirt,Start,1/4,1/2,Str,,4
0,SUN,1,12,2019,4,Thoroughbred,SixAndOneHalfFurlongs,Dirt,Start,1/4,1/2,Str,,4
0,SUN,1,12,2019,5,Thoroughbred,SixFurlongs,Dirt,Start,1/4,1/2,Str,,4
0,SUN,1,12,2019,6,Thoroughbred,SixFurlongs,Dirt,Start,1/4,1/2,Str,,4
0,SUN,1,12,2019,7,Thoroughbred,OneMileAndSeventyYards,Dirt,Start,1/4,1/2,3/4,Str,5


In [ ]:
"""
need to create parsers for:
    betting stats
    running line stats
    trainers/owners
    dunzo
"""

In [ ]:
#line = 'LOSALAMIT?OS-January4,2019-Race1' #first
#line = 'MAIDENCLAIMING-Thoroughbred' #second
#line = 'Four And One Half Furlongs On The Dirt Track Record: (Valiant Pete - 49.20 - August 11, 1990)' #fifth
#line = 'Copyright 2020 Equibase Company LLC. All Rights Reserved.' #race flag
#line = '    Weather:   Clear Track:Fast' #ninth
#line = '    Off at: 7:02 Start: Good for all' #tenth
#line = '   Last Raced       Pgm HorseName(Jockey)                        WgtM/E PP 1/4        1/2       1m       11/4     Str      Fin     Odds Comments' #belmont
line = '    Last Raced            Pgm HorseName(Jockey)                                      Wgt M/E PP Start             1/4         Str         Fin       Odds Comments' #reg